# Case Study 1 - Danny's Dinner

[https://8weeksqlchallenge.com/case-study-1/](https:\8weeksqlchallenge.com\case-study-1\)

Case Study Questions Each of the following case study questions can be answered using a single SQL statement:

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

Bonus Questions Join All The Things The following questions are related creating basic data tables that Danny and his team can use to quickly derive insights without needing to join the underlying tables using SQL.

Recreate the following table output using the available data:

| **customer\_id** | **order\_date** | **product\_name** | **price** | **member** |
| --- | --- | --- | --- | --- |
| A | 2021-01-01 | curry | 15 | N |
| A | 2021-01-01 | sushi | 10 | N |
| A | 2021-01-07 | curry | 15 | Y |
| A | 2021-01-10 | ramen | 12 | Y |
| A | 2021-01-11 | ramen | 12 | Y |
| A | 2021-01-11 | ramen | 12 | Y |
| B | 2021-01-01 | curry | 15 | N |
| B | 2021-01-02 | curry | 15 | N |
| B | 2021-01-04 | sushi | 10 | N |
| B | 2021-01-11 | sushi | 10 | Y |
| B | 2021-01-16 | ramen | 12 | Y |
| B | 2021-02-01 | ramen | 12 | Y |
| C | 2021-01-01 | ramen | 12 | N |
| C | 2021-01-01 | ramen | 12 | N |
| C | 2021-01-07 | ramen | 12 | N |

Rank All The Things Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

| **customer\_id** | **order\_date** | **product\_name** | **price** | **member** | **ranking** |
| --- | --- | --- | --- | --- | --- |
| A | 2021-01-01 | curry | 15 | N | null |
| A | 2021-01-01 | sushi | 10 | N | null |
| A | 2021-01-07 | curry | 15 | Y | 1 |
| A | 2021-01-10 | ramen | 12 | Y | 2 |
| A | 2021-01-11 | ramen | 12 | Y | 3 |
| A | 2021-01-11 | ramen | 12 | Y | 3 |
| B | 2021-01-01 | curry | 15 | N | null |
| B | 2021-01-02 | curry | 15 | N | null |
| B | 2021-01-04 | sushi | 10 | N | null |
| B | 2021-01-11 | sushi | 10 | Y | 1 |
| B | 2021-01-16 | ramen | 12 | Y | 2 |
| B | 2021-02-01 | ramen | 12 | Y | 3 |
| C | 2021-01-01 | ramen | 12 | N | null |
| C | 2021-01-01 | ramen | 12 | N | null |
| C | 2021-01-07 | ramen | 12 | N | null |

<span style="color: var(--vscode-foreground);">1. What is the total amount each customer spent at the restaurant?</span>  

- **INNER JOIN** `sales` with `menu` to obtain price of each product.
- **GROUP BY** `customer_id` and use **SUM** to aggregate total price

In [3]:
SELECT sales.customer_id,
    SUM(menu.price) AS total_price
FROM sales 
INNER JOIN menu
    ON sales.product_id = menu.product_id
GROUP BY sales.customer_id;

(3 rows affected)

Total execution time: 00:00:00.030

customer_id,total_price
A,76
B,74
C,36


<span style="color: var(--vscode-foreground);">2. How many days has each customer visited the restaurant?</span>  
- **GROUP BY** `customer_id`
- **COUNT** and **DISTINCT** to count unique dates

In [4]:
SELECT customer_id, 
    COUNT(DISTINCT(order_date)) AS total_days_visited
FROM sales
GROUP BY customer_id;

(3 rows affected)

Total execution time: 00:00:00.020

customer_id,total_days_visited
A,4
B,6
C,2


<span style="color: var(--vscode-foreground);">2. What was the first item from the menu purchased by each customer?</span>

- Construct CTE (`temp_table`) with customer details, product information, and ranking
    - **INNER JOIN** `sales` with `menu` on `sales.product_id = menu.product_id`
        - Combines `sales` and `menu` tables to add `product_name` to sales data.
    - Use **DENSE_RANK()** window function
        - **PARTITION BY** `customer_id` to create a ranking segment for each customer.
        - **ORDER BY** `order_date` to determine the rank based on the order date.
        - Assign the rank as `product_rank`.
- Use CTE in the main query to filter and display specific records
    - Select `customer_id`, `order_date`, and `product_name` from `temp_table`.
    - Apply **WHERE** condition to filter the results
        - Condition: `product_rank = 1` to show only the first ranked product for each customer.


In [5]:
WITH temp_table AS (SELECT customer_id,
                           order_date,
                           sales.product_id,
                           product_name,
                           DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY order_date) AS product_rank
                    FROM sales
                         INNER JOIN menu ON sales.product_id = menu.product_id)
SELECT customer_id,
       order_date,
       product_name
FROM temp_table
WHERE product_rank = 1;

(5 rows affected)

Total execution time: 00:00:00.019

customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-01,curry
C,2021-01-01,ramen
C,2021-01-01,ramen


<span style="color: var(--vscode-foreground);">4. What is the most purchased item on the menu and how many times was it purchased by all customers?</span>
- Construct CTE (`temp_table`) with `customer_id`, `order_date`, `product_id`, `product_name`, and `price`
    - **INNER JOIN** `sales` with `menu` on `sales.product_id = menu.product_id`
        - This join combines the `sales` and `menu` tables to match records based on `product_id`.
    - Select `customer_id`, `order_date`, `sales.product_id`, `product_name`, and `price` from the joined tables.
- Use CTE in the main query to identify the most frequently ordered product
    - **SELECT TOP (1)** to retrieve the single most ordered product.
    - **COUNT(product_id) AS product_count** to calculate the total number of times each product has been ordered.
    - **GROUP )te the results by `product_name`.
    - **ORDER BYt DESC** to sort the products in descending order of their count, ensuring the most frequently ordered product is listed first.


In [1]:
WITH temp_table AS (SELECT customer_id,
                          order_date,
                          sales.product_id,
                          product_name,
                          price
                   FROM sales
                        INNER JOIN menu ON sales.product_id = menu.product_id)
SELECT TOP (1) product_name,
               COUNT(product_id) AS product_count
FROM temp_table
GROUP BY product_name
ORDER BY product_count DESC;

(1 row affected)

Total execution time: 00:00:00.114

product_name,product_count
ramen,8


<span style="color: var(--vscode-foreground);">5. Which item was the most popular for each customer?</span>
- Construct a Common Table Expression (CTE) named `temp_table` with customer ID, product name, product count, and product ranking
    - **INNER JOIN** `sales` with `menu` on `sales.product_id = menu.product_id`
        - This join combines the `sales` and `menu` tables based on matching `product_id`, enabling access to `product_name`.
    - **COUNT(menu.product_name)** to calculate the frequency of each product ordered by each customer.
    - Use **DENSE_RANK()** window function
        - **PARTITION BY** `sales.customer_id` to rank products within each customer's orders.
        - **ORDER BY** `COUNT(sales.customer_id) DESC` to rank products based on their order frequency for each customer, with most frequent products ranked higher.
    - **GROUP BY** `customer_id` and `menu.product_name` to aggregate data for the ranking.
- Use CTE in the main query to display the most popular product per customer
    - Select `customer_id`, `product_name`, `product_count`, and `product_rank` from `most_pop`.
    - Apply a **WHERE** clause
        - Condition: `product_rank = 1` to filter and show only the top-ranked products for each customer (i.e., the most popular products per customer).


In [1]:
WITH most_pop AS (SELECT customer_id,
                         menu.product_name,
                         COUNT(menu.product_name) AS product_count,
                         DENSE_RANK() OVER (PARTITION BY sales.customer_id ORDER BY COUNT(sales.customer_id) DESC) AS product_rank
                  FROM sales
                       INNER JOIN menu ON sales.product_id = menu.product_id
                  GROUP BY customer_id, menu.product_name)
SELECT customer_id,
       product_name,
       product_count,
       product_rank
FROM most_pop
WHERE product_rank = 1;

(5 rows affected)

Total execution time: 00:00:00.051

customer_id,product_name,product_count,product_rank
A,ramen,3,1
B,sushi,2,1
B,curry,2,1
B,ramen,2,1
C,ramen,3,1


<span style="color: var(--vscode-foreground);">6. Which item was purchased first by the customer after they became a member?</span>

In [2]:
WITH first_item AS (SELECT sales.customer_id,
                           sales.order_date,
                           menu.product_name,
                           DENSE_RANK() OVER (PARTITION BY sales.customer_id ORDER BY sales.order_date ASC) AS rank
                    FROM sales
                         INNER JOIN members ON sales.customer_id = members.customer_id
                        AND sales.order_date > members.join_date
                         INNER JOIN menu ON sales.product_id = menu.product_id)
SELECT customer_id,
       order_date,
       product_name
FROM first_item
WHERE rank = 1;


(2 rows affected)

Total execution time: 00:00:00.314

customer_id,product_name
A,ramen
B,sushi


<span style="color: var(--vscode-foreground);">7. Which item was purchased just before the customer became a member?</span>

In [3]:
WITH first_item AS (SELECT sales.customer_id,
                           sales.order_date,
                           menu.product_name,
                           DENSE_RANK() OVER (PARTITION BY sales.customer_id ORDER BY sales.order_date DESC) AS rank
                    FROM sales
                         INNER JOIN members ON sales.customer_id = members.customer_id
                        AND sales.order_date < members.join_date
                         INNER JOIN menu ON sales.product_id = menu.product_id)
SELECT customer_id,
       order_date,
       product_name
FROM first_item
WHERE rank = 1;

(3 rows affected)

Total execution time: 00:00:00.070

customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


<span style="color: var(--vscode-foreground);">8. What is the total items and amount spent for each member before they became a member?</span>

In [4]:
WITH temp_table AS (SELECT sales.customer_id,
                           sales.order_date,
                           menu.product_id,
                           menu.price
                    FROM sales
                         INNER JOIN members ON sales.customer_id = members.customer_id
                        AND sales.order_date < members.join_date
                         INNER JOIN menu ON sales.product_id = menu.product_id)
SELECT customer_id,
       COUNT(product_id) AS total_items,
       SUM(price) AS total_price
FROM temp_table
GROUP BY customer_id;

(2 rows affected)

Total execution time: 00:00:00.258

customer_id,total_items,total_price
A,2,25
B,3,40


<span style="color: var(--vscode-foreground);">9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?</span>

In [1]:
WITH temp_table AS (SELECT sales.customer_id,
                           SUM(menu.price) total_price,
                           SUM(IIF(menu.product_id = 1, 1, 0)) AS sushi
                    FROM sales
                         INNER JOIN menu ON sales.product_id = menu.product_id
                    GROUP BY sales.customer_id)
SELECT customer_id,
       CASE
           WHEN sushi > 0 THEN total_price * 2
           ELSE total_price
           END AS points
FROM temp_table;

(3 rows affected)

Total execution time: 00:00:00.101

customer_id,points
A,152
B,148
C,36


10\. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?